# Fractional Cover Demo

This demonstrates how to access large continental scale raster data without downloading everything.

* Metadata: https://portal.tern.org.au/metadata/TERN/0997cb3c-e2e2-45be-ac82-f5e13d24331c
* Map Visualiser: https://maps.tern.org.au/map/0997cb3c-e2e2-45be-ac82-f5e13d24331c
* Data Access: https://data.tern.org.au/gov/qld/fractional_cover_v3/landsat/fractional_cover/seasonal/

Landsat based fractional cover data comes ~30m resolution, with 3 bands per pixel

* band 1: bare ground fraction (in percent)
* band 2: green vegetation fraction (in percent)
* band 3: non-green vegetation fraction (in percent).

The dataset is a moasic raster datasets, with one GeoTiff file per state, and quarterly time steps since 1987.
All single tiff files for each time step are combined in a Virtual Raster Dataset (GDAL VRT file).

This demo utilises the STAC catalog to find all available VRT files and loads them into a single data array.

The data is then sliced to the bounding box of Brisbane LGA and summarised to produce a time series.

* STAC Collection: https://data.tern.org.au/gov/qld/fractional_cover_v3/landsat/fractional_cover/seasonal/collection.json

⚠️ Important: To access data programmatically, you need to create an TERN API key: https://account.tern.org.au/

Once you created an api key, create a .netrc file in your home directory (cd ~, vim .netrc), then copy the following into the file, replacing YOUR_REAL_API_KEY with your key copied from https://account.tern.org.au/

    machine data.tern.org.au
        login apikey
        password YOUR_REAL_API_KEY

In [1]:
# set to true to enable hvplots (disabled to keep .ipynb file size small)
USE_HV_PLOT = False

import pystac
import geopandas as gpd
import xarray as xr
import dask.array as da
import rioxarray as rxr
if USE_HV_PLOT:
    import hvplot.xarray
import numpy as np
from dask.distributed import LocalCluster, Client
import matplotlib.pyplot as plt

In [2]:
import os 
# This a small optimisaton that instructs GDAL to not search for auxillary files
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"

In [3]:
# start a local DASK cluster, for multiprocessing
cluster = LocalCluster()
client = Client(cluster)

In [4]:
# get region of interest as a GeoPandas DataFrame - Brisbane LGA
region = gpd.read_file("https://geoserver.tern.org.au/geoserver/administrative_area/ows?service=WFS&version=2.0.0&request=GetFeature&typename=administrative_area%3Aauslga_2023&outputFormat=application/json&cql_filter=lga_name23='Brisbane'")
# reproject to EPSG:3577 (I now that already)
region = region.to_crs("EPSG:3577")

In [5]:
region.explore()

# Load STAC Collection

In [6]:
# Get Fractional Cover Collection 
# Open static catalog and all items in it 
collection = pystac.Collection.from_file("https://data.tern.org.au/gov/qld/fractional_cover_v3/landsat/fractional_cover/seasonal/collection.json")
items = list(collection.get_all_items())

In [7]:
%%time
# open all vrts and combine into a single xarray dataset along a new time dimension
def generate_datasets(items):
    datasets = []
    for item in items:
        vrt_url = list(item.get_assets(media_type="application/xml", role="data").values())[0].get_absolute_href()
        ds = rxr.open_rasterio(vrt_url, chunks=True)
        # ds = ds.where(ds != ds.rio.nodata, np.nan)
        # assign timestep for new time coordinate
        datasets.append(ds.assign_coords(time=item.properties['datetime']))
    return datasets
        

ds = xr.concat(
    # skip the first few time steps as they have different bounds
    generate_datasets(sorted(items, key=lambda x: x.properties['datetime'])[3:]),
    dim="time",
)

/mnt/home/g.weis_uq/.venv/ridemo/lib/python3.10/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(
/mnt/home/g.weis_uq/.venv/ridemo/lib/python3.10/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(
/mnt/home/g.weis_uq/.venv/ridemo/lib/python3.10/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(
/mnt/home/g.weis_uq/.venv/ridemo/lib/python3.10/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/mnt/home/g.weis_uq/.venv/ridemo/lib/python3.10/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/mnt/home/g.weis_uq/.venv/ridemo/lib/python3.10/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks 

CPU times: user 35.4 s, sys: 2.39 s, total: 37.8 s
Wall time: 58 s


# Inspect dataset

In [8]:
ds

<xarray.DataArray (time: 147, band: 3, y: 135159, x: 141481)> Size: 34TB
dask.array<concatenate, shape=(147, 3, 135159, 141481), dtype=float32, chunksize=(1, 1, 10842, 10721), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 24B 1 2 3
  * x            (x) float64 1MB -1.945e+06 -1.945e+06 ... 2.3e+06 2.3e+06
  * y            (y) float64 1MB -4.91e+06 -4.91e+06 ... -8.555e+05 -8.554e+05
    spatial_ref  int64 8B 0
  * time         (time) <U20 12kB '1988-09-01T00:00:00Z' ... '2025-03-01T00:0...
Attributes:
    _FillValue:    255
    scale_factor:  1.0
    add_offset:    0.0

# Apply study area

In [9]:
# slice dataset by region bounds
ds_bris = ds.sel(
    x=slice(region.bounds.minx.item(), region.bounds.maxx.item()), 
    y=slice(region.bounds.miny.item(), region.bounds.maxy.item())
)
ds_bris

<xarray.DataArray (time: 147, band: 3, y: 2130, x: 2749)> Size: 10GB
dask.array<getitem, shape=(147, 3, 2130, 2749), dtype=float32, chunksize=(1, 1, 2130, 2749), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 24B 1 2 3
  * x            (x) float64 22kB 2.012e+06 2.012e+06 ... 2.094e+06 2.094e+06
  * y            (y) float64 17kB -3.169e+06 -3.169e+06 ... -3.105e+06
    spatial_ref  int64 8B 0
  * time         (time) <U20 12kB '1988-09-01T00:00:00Z' ... '2025-03-01T00:0...
Attributes:
    _FillValue:    255
    scale_factor:  1.0
    add_offset:    0.0

# Produce Average Time series over study area

Using the dataset like above was really slow for some reason, so let's try another approach and build the dataset by applying the transformation we want straight away.

In [10]:
%%time
# open all vrts and combine into a single xarray dataset along a new time dimension
def generate_datasets_2(items):
    datasets = []
    for item in items:
        vrt_url = list(item.get_assets(media_type="application/xml", role="data").values())[0].get_absolute_href()
        ds = rxr.open_rasterio(vrt_url, chunks=True)
        # slice to study area
        ds = ds.sel(
            x=slice(region.bounds.minx.item(), region.bounds.maxx.item()), 
            y=slice(region.bounds.maxy.item(), region.bounds.miny.item())
        )
        # convert no data to NaN
        ds = ds.where(ds != ds.rio.nodata, np.nan)
        # apply mean and compute straight away
        ds = ds.mean(dim=("x", "y"), skipna=True)
        # assign timestep for new time coordinate
        datasets.append(ds.assign_coords(time=item.properties['datetime']))
    return datasets
        

ds2 = xr.concat(
    # skip the first few time steps as they have different bounds
    generate_datasets_2(sorted(items, key=lambda x: x.properties['datetime'])[3:]),
    dim="time",
)

CPU times: user 6.32 s, sys: 145 ms, total: 6.47 s
Wall time: 6.16 s


In [11]:
%%time
# This step actually execute data fetch and computation. Dask will take care of parallelising where it makes sense.
data_bris = ds2.compute()

CPU times: user 14.9 s, sys: 1.83 s, total: 16.7 s
Wall time: 3min 11s


In [12]:
# band 0 ... bare ground 
# band 1 ... green vegetation 
# band 2 ... non-green vegetation
with plt.ioff():
    lines = data_bris.plot.line(x="time", hue="band", figsize=(12, 8))
    plt.savefig("images/fractional_time.png");
    plt.close()

![Fractional Cover Time series](images/fractional_time.png)

In [13]:
# re-label bands for plotting
if USE_HV_PLOT:
    data_labeled = data_bris.assign_coords(band=["Bare", "Green", "Non-Green"])
    data_labeled.hvplot.line(width=1000, height=600, groupby="band", label="Band").overlay()